In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import numpy as np
import torchvision
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()

In [2]:
filename_list = []
for root, dirs, files in os.walk("C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/images"):
    for file in files:
        if file.endswith(".jpg"):
            name = file
            name = name.replace(".jpg", '')
            name = name.split('_')[:-1]
            name = ' '.join(name)
            filename_list.append(name)

In [3]:
unique_name = []
for item in filename_list:
    if item not in unique_name:
        unique_name.append(item)
dog_list = []
cat_list = []
for item in unique_name:
    if item[0].isupper():
        cat_list.append(item)
    else:
        dog_list.append(item)

In [4]:
##format data in folder
#import shutilfor root, dirs, files in os.walk("C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/images"):
#
#        r = np.random.rand()
#    for file in files:
#            if r > 0.92:
#        if file[0].isupper():
#                            'C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/data_2_categories/test/' + 'cat')
#                shutil.move("C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/images/" + file, 
#                shutil.move("C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/images/" + file, 
#            elif r <= 0.72:
#            else:
#                            'C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/data_2_categories/train/' + 'cat')
#                            'C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/data_2_categories/val/' + 'cat')
#                shutil.move("C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/images/" + file, 
#            if r > 0.92:
#        else:
#                            'C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/data_2_categories/test/' + 'dog')
#                shutil.move("C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/images/" + file, 
#                shutil.move("C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/images/" + file, 
#            elif r <= 0.72:
#            else:
#                            'C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/data_2_categories/train/' + 'dog')
#                            'C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/data_2_categories/val/' + 'dog')
#                shutil.move("C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/images/" + file, 

In [5]:
#dir = "C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/data_2_categories/val/cat"
#for root, dirs, files in os.walk(dir):
#    for file in files:
#        shutil.move(dir + '/' + file, 'C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/images')

In [4]:
#laod data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = 'C:/Users/Annika Wong/OneDrive/KTH/2022-2023Spring_P4/DD2424/project/oxford-iiit-pet/data_2_categories'
train_datesets = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
train_loaders = torch.utils.data.DataLoader(train_datesets, batch_size=4, shuffle=True, num_workers=4)
train_dataset_sizes = len(train_datesets)

val_datesets = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val'])
val_loaders = torch.utils.data.DataLoader(val_datesets, batch_size=4, shuffle=True, num_workers=4)
val_dataset_sizes = len(val_datesets)

test_datesets = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])
test_loaders = torch.utils.data.DataLoader(test_datesets, batch_size=4, shuffle=True, num_workers=4)
test_dataset_sizes = len(test_datesets)

class_name = train_datesets.classes
device = torch.device("cpu") #torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [6]:
#from torchvision.utils import make_grid
#
#def show_images(images, nmax=64):
#    fig, ax = plt.subplots(figsize=(8, 8))
#    ax.set_xticks([]); ax.set_yticks([])
#    ax.imshow(make_grid((images.detach()[:nmax]), nrow=8).permute(1, 2, 0))
#def show_batch(dl, nmax=64):
#    for images in dl:
#        show_images(images, nmax)
#        break
#
#show_batch(test_loaders)

: 

: 

In [10]:
def training(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_para = copy.deepcopy(model.state_dict())
    best_accuracy = 0

    for epoch in range(num_epochs):
        print(f'Current epoch: {epoch}')

        ###################### TRAINING #######################
        model.train()

        running_loss = 0
        accuracy = 0

        temp_count = 0
        for batch, label in train_loaders:
            if temp_count % 100 == 0:
                print(f'Current batch {temp_count}')
            temp_count += 1

            batch = batch.to(device)
            label = label.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                # Forward
                output = model(batch)

                _, preds = torch.max(output, 1)
                loss = criterion(output, label)

                #Backward
                loss.backward()
                
                optimizer.step()

            running_loss += loss.item() * batch.size(0)
            accuracy += torch.sum(preds == label.data)

            scheduler.step()

        epoch_loss = running_loss/train_dataset_sizes
        epoch_accuracy = accuracy/train_dataset_sizes
        print(f'Total train image {temp_count}')
        print(f'Training Loss: {epoch_loss} Accuracy: {epoch_accuracy}')

        ###################### VALIDATION #######################
        model.eval()

        running_loss = 0
        accuracy = 0

        temp_count = 0

        for batch, label in val_loaders:
            if temp_count % 100 == 0:
                print(f'Current image {temp_count}')
            temp_count += 1

            batch = batch.to(device)
            label = label.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(False):
                #Forward
                output = model(batch)
                _, preds = torch.max(output, 1)
                loss = criterion(output, label)
            
            running_loss += loss.item() * batch.size(0)
            accuracy += torch.sum(preds == label.data)

        epoch_loss = running_loss/val_dataset_sizes
        epoch_accuracy = accuracy/val_dataset_sizes

        print(f'Total val image {temp_count}')
        print(f'Validation Loss: {epoch_loss} Accuracy: {epoch_accuracy}')

        if epoch_accuracy > best_accuracy:
            best_accuracy = epoch_accuracy
            best_model_para = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_para)

    ###################### TEST #######################

    model.eval()

    running_loss = 0
    accuracy = 0
    
    for batch, label in test_loaders:
        batch = batch.to(device)
        label = label.to(device)

        #optimizer.zero_grad()

        #Forward
        output = model(batch)
        _, preds = torch.max(output, 1)
        loss = criterion(output, label)
        
        running_loss += loss.item() * batch.size(0)
        accuracy += torch.sum(preds == label.data)

    test_loss = running_loss/test_dataset_sizes
    test_accuracy = accuracy/test_dataset_sizes

    print(f'Test Loss: {test_loss} Accuracy: {test_accuracy}')

    return model

In [13]:
model_tuned = models.resnet18(weights='IMAGENET1K_V1')
num_features = model_tuned.fc.in_features

output_size = 2
model_tuned.fc = nn.Linear(num_features, output_size) #cat and dog

model_tuned = model_tuned.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_model = optim.Adam(model_tuned.parameters())

lr_scheduler_model = lr_scheduler.StepLR(optimizer_model, 5)

In [14]:
model_tuned = training(model_tuned, criterion, optimizer_model, lr_scheduler_model, num_epochs=1)

Current epoch: 0
Current batch 0
Current batch 100
Current batch 200
Current batch 300
Current batch 400
Current batch 500
Current batch 600
Current batch 700
Current batch 800
Current batch 900
Current batch 1000
Current batch 1100
Current batch 1200
Current batch 1300
Total train image 1328
Training Loss: 0.9565999647304275 Accuracy: 0.6116760969161987
Current image 0
Current image 100
Current image 200
Current image 300
Total val image 363
Validation Loss: 1.3688893690142907 Accuracy: 0.6300897002220154
Test Loss: 1.2323078618882293 Accuracy: 0.6244056820869446


In [15]:
model_tuned = training(model_tuned, criterion, optimizer_model, lr_scheduler_model, num_epochs=25)

Current epoch: 0
Current batch 0
Current batch 100
Current batch 200
Current batch 300
Current batch 400
Current batch 500
Current batch 600
Current batch 700
Current batch 800
Current batch 900
Current batch 1000
Current batch 1100
Current batch 1200
Current batch 1300
Total train image 1328
Training Loss: 0.9561071458860193 Accuracy: 0.6233521699905396
Current image 0
Current image 100
Current image 200
Current image 300
Total val image 363
Validation Loss: 1.319199910549366 Accuracy: 0.6293995976448059
Current epoch: 1
Current batch 0
Current batch 100
Current batch 200
Current batch 300
Current batch 400
Current batch 500
Current batch 600
Current batch 700
Current batch 800
Current batch 900
Current batch 1000
Current batch 1100
Current batch 1200
Current batch 1300
Total train image 1328
Training Loss: 0.9635523935383548 Accuracy: 0.6169491410255432
Current image 0
Current image 100
Current image 200
Current image 300
Total val image 363
Validation Loss: 1.4208392315486984 Accura

KeyboardInterrupt: 

In [20]:
import numpy as np

lambda_ = np.linspace(0.0002, 0.0006, 5)
eta_max = np.linspace(1e-1, 5e-1, 5)
cycles = np.linspace(3,7,5)

In [21]:
print(cycles)
print(eta_max)
print(lambda_)

[3. 4. 5. 6. 7.]
[0.1 0.2 0.3 0.4 0.5]
[0.0002 0.0003 0.0004 0.0005 0.0006]
